# VBA Benchmark Model

In [1]:
import numpy
import xlrd
from openpyxl import load_workbook
from vbaFunc import jumpoff, xTFR, clear_Input
from vbaFunc import readCNAPC, readSYI, readSATP, readERP, readTFR, readASFR, readLEMS, readMASMR
from vbaFunc import inputASFR, inputbirth, inputASDR, inputdeath, inputMigration, writeAccount, writeSAI

# Read in Excel Workbook (both Readable and Writable)
wt_loc = '../VBA_Benchmark/vba.xlsx'
rd_loc = '../VBA_Benchmark/original.xlsm'
wb_rd = xlrd.open_workbook(rd_loc)
wb_wt = load_workbook(wt_loc)

## xTRF

#### Generating Fertility Data

In [2]:
# Read in Sheets
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
wb_wt_fertility = wb_wt["Fertility"]

# Define variables
numareas = int(sheet_label.cell_value(2256, 0))
numages = 18
sextypes = 2
set_year_female = 7
set_year_male = 5

# Create empty array for storing jumpoffERP and xTFR results
jumpoffERP = numpy.zeros((numareas, sextypes, numages))
jumpoffERP[:] = numpy.nan
result_xTFR = numpy.zeros((numareas))
result_xTFR[:] = numpy.nan

# Generate jumpoffERP for further calculation
jumpoffERP = jumpoff(numareas, numages, jumpoffERP, sheet_agesex, set_year_female, set_year_male)

# Generate xTFR vector for storing
result_xTFR = xTFR(jumpoffERP, result_xTFR, numareas)

# Store into Sheet 'Fertility' for further usage
row = 7
column = 4
for i in range(len(result_xTFR)):
    wb_wt_fertility.cell(row, column).value = result_xTFR[i]
    row += 1
wb_wt.save(wt_loc)

## PrepareData

#### Clear Input Data

In [3]:
# Before running the PrepareData Function, clear the input data if any
wb_wt_SmallAreaInputs = wb_wt["SmallAreaInputs"]
wb_wt_Accounts = wb_wt["Accounts"]

# Clear the data and save the writable workbook
clear_Input(wb_wt_SmallAreaInputs)
clear_Input(wb_wt_Accounts)
wb_wt.save(wt_loc)

#### Load in Required Data for Generating Input Data

In [4]:
# Read in readable required data for generating input data for further usage
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_SmallAreaTotals = wb_rd.sheet_by_name('SmallAreaTotals')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
sheet_mortality = wb_rd.sheet_by_name('Mortality')
sheet_migration = wb_rd.sheet_by_name('Migration')

# Temeorary read of new generated fertility
temp_wb_rd = xlrd.open_workbook(wt_loc)
sheet_fertility = temp_wb_rd.sheet_by_name('Fertility')

#### Start Creating Input Data

##### Define Variables

In [5]:
numages = 18                                    # Number of age groups in ERPs
lastage = 17                                    # Number of age groups in projections output
nLxMS = 21                                      # nLx mortality surface groups
age_ASFR = 7                                    # Age groups amount in ASFRs Model
numareas = int(sheet_label.cell_value(2256, 0)) # Number of small areas
final = int(sheet_label.cell_value(2271, 0))    # Final projection year number
SRB = sheet_fertility.cell_value(2270, 1)       # Assumed Sex Ratio at Birth (Male : Female)
modelTFR  = 0                                   # Model TFR Parameter
for i in range(2259, 2266): modelTFR += sheet_fertility.cell_value(i, 1)
modelTFR *= 5

##### Read in Data for Generating Input Data

In [6]:
## All two dimensional array with 0 and 1 as sex index, we treat 0 as female and 1 as male
# Read in AreaCode, AreaName, AgeGroup, Period-Cohort Labels
Areacode, Areaname, agelabel, pclabel = readCNAPC(numareas, numages, lastage, sheet_label)

# Read in Sex, Year, Interval Labels
sexlabel, yearlabel, intervallabel = readSYI(final, sheet_label)

# Read in Estimated & Projected Small Area Total Population (Aggregated)
TotPop = readSATP(2, sheet_SmallAreaTotals, numareas)

# Read in small area ERP (Estimated Resident Population) by age & sex (before and at jumpoff year)
ERP = readERP(2, numareas, numages, sheet_agesex)

# Area-specific TFRs
TFR = readTFR(final, numareas, sheet_fertility)

# Read in data for ASFRs and Preliminary ASFRs (Area specific Fertility Rate)
modelASFR, prelimASFR = readASFR(numareas, sheet_fertility, age_ASFR)

# Read in data for Life Expectancy and Mortality Surface
eO, MS_nLx, MS_TO = readLEMS(final, numareas, lastage, nLxMS, sheet_mortality)

# Read in data of migration data and for ASMRs (Area specific Mortality Rate)
totmig, modelASMR = readMASMR(numareas, lastage, sheet_migration)

##### Some Checks

In [7]:
# TFRs > 0
for i in range(final + 1):
    for a in range(numareas):
        if (TFR[i, a] < 0.1):
            print("Error: Year = ", yearlabel[i], " & Area = ", Areaname[a])
            break
    break

# eOs > 0
for i in range(final + 1):
    for a in range(numareas):
        for q in range(2):
            if (eO[i, a, q] < 0.1):
                print("Error: Year = ", yearlabel[i], " & Area = ", Areaname[a], " & Sex = ", sexlabel[q])
                break
        break
    break

# Check Migration Turnover > 0
for i in range(numareas):
    if (totmig[i] < 0.1):
        print("Error: Area =", Areaname[i])
    break
    
# Check Small Area Population Total > 0
for i in range(2):
    for a in range(numareas):
        if (TotPop[i, a] < 0.1):
            print("Error Year =", yearlabel[i], " & Area = ", Areaname[a])
        break
    break

##### Calculate Input Data

In [8]:
# Create input data for ASFR (Area specific Fertility Rate)
ASFR = inputASFR(final, numareas, age_ASFR, sheet_fertility, prelimASFR, modelASFR, TFR, modelTFR)

# Estimate base period birth by sex for the population 'Accounts'
tempbirthssex, temptotbirths = inputbirth(numareas, age_ASFR, ASFR, ERP, SRB)

# Calculate base period projected age-specific death rate (used in estimating death account)
ASDR = inputASDR(final, numareas, lastage, nLxMS, eO, MS_TO, MS_nLx)

# Calculate estimated death for age-sex cohorts and total population accounts
tempdeaths, temptotdeaths = inputdeath(numareas, lastage, ASDR, ERP)

# Calculate migration input data
ASOMR, inward, outward, prelimmig, scaledmig, prelimbaseinward, prelimbaseoutward, cohortnetmig, adjnetmig, scalingfactor2a, scalingfactor2b = \
inputMigration(final, numareas, lastage, modelASMR, ERP, totmig, TotPop, temptotdeaths, temptotbirths, tempdeaths, tempbirthssex, ASDR, Areaname, sexlabel, pclabel, intervallabel)

##### Write Input Data

In [9]:
# Write Input Data to Accounts Sheet
wb_wt_Accounts = writeAccount(wb_wt_Accounts, numareas, Areaname, lastage, sexlabel, pclabel, tempbirthssex, inward, outward, prelimmig, scaledmig, prelimbaseinward, prelimbaseoutward, cohortnetmig, adjnetmig, scalingfactor2a, scalingfactor2b, ERP, tempdeaths)

# Write Input Data to SmallAreaInputs Sheet
wb_wt_SmallAreaInputs = writeSAI(wb_wt_SmallAreaInputs, intervallabel, numareas, lastage, final, Areacode, Areaname, pclabel, ASDR, ASOMR, inward, agelabel, ASFR)

# Save the written workbook
wb_wt.save(wt_loc)
print("Input data is prepared")

Input data is prepared
